In [2]:
import sys
sys.path.append("..")

import numpy as np
from scipy.optimize import minimize

from pyscf import gto
from pyscf.scf import hf
from pyscf.fci import cistring, direct_spin1, FCI

from dmet.model import hubbard
from dmet.main.solver import DMET_SCF, DMET_FCI
from dmet.main.outer import DMET
from cpskit.fci import CPSMethod, ci_cps


In [11]:
# Generate basis sets given 1-site Gutzwiller correlators

def _get_projected_basis_1site(siteidx, ci0, norb, nelec):

    num_string = cistring.num_strings(norb, nelec//2)
    if ci0 is 0: ci0 = np.random.random((num_string, num_string))
    strs = cistring.make_strings(range(norb), nelec//2)

    occidx = np.where(strs & 2**siteidx == 2**siteidx)[0]
    viridx = np.where(strs & 2**siteidx == 0)[0]

    ci_ud = np.zeros_like(ci0)
    ci_ud[np.ix_(occidx, occidx)] += ci0[np.ix_(occidx, occidx)]

    ci_u0 = np.zeros_like(ci0)
    ci_u0[np.ix_(occidx, viridx)] += ci0[np.ix_(occidx, viridx)]

    ci_0d = np.zeros_like(ci0)
    ci_0d[np.ix_(viridx, occidx)] += ci0[np.ix_(viridx, occidx)]

    ci_00 = np.zeros_like(ci0)
    ci_00[np.ix_(viridx, viridx)] += ci0[np.ix_(viridx, viridx)]    

    return ci_ud, ci_u0, ci_0d, ci_00

def trans_energy(h1e, eri, cibra, ciket, norb, nelec, link_index=None):
    '''Compute the FCI electronic energy for given Hamiltonian and FCI vector.
    '''
    h2e = direct_spin1.absorb_h1e(h1e, eri, norb, nelec, .5)
    ci1 = direct_spin1.contract_2e(h2e, ciket, norb, nelec, link_index)
    return np.dot(cibra.reshape(-1), ci1.reshape(-1))

def gwfexact_grad_1site(ci0, norb, nelec, proj, h1e, eri):

    fraglst_dg = [[[i] for i in range(1,norb)]]
    fraglst_full = [[[i] for i in range(norb)]]

    ci_g = ci_cps(norb, (nelec//2,nelec//2), ci0, 
    [proj], fraglst_full, normalized=False)
    ci_dg = ci_cps(norb, (nelec//2,nelec//2), ci0,
    [proj], fraglst_dg, normalized=False)
    ci_dgs = _get_projected_basis_1site(0, ci_dg, norb, nelec)

    trans_Hs = [trans_energy(h1e, eri, cibra, ci_g, norb, nelec) for cibra in ci_dgs]
    trans_Ss = [np.dot(cibra.reshape(-1), ci_g.reshape(-1)) for cibra in ci_dgs]

    partial_G = [trans_Hs[i] * (2*norb) for i in range(4)]

    return np.array(partial_G)

def gwfexact_1site(ci0, norb, nelec, proj, h1e, eri):
    fraglst_full = [[[i] for i in range(norb)]]
    ci_g = ci_cps(norb, (nelec//2,nelec//2), ci0, 
    [proj], fraglst_full, normalized=False)

    H = direct_spin1.energy(h1e, eri, ci_g, norb, nelec)
    return H

def get_gsbasis(ci0, proj, norb, nelec):
    
    gsbasis = []
    gsbasis.append([ci0])
    for i in range(1, 4):
        occs = cistring._gen_occslst(range(norb), i)
        basislst = []
        for occ in occs:
            basislst.append(ci_cps(norb, (nelec//2, nelec//2), 
            ci0, [proj], [[[x] for x in occ]], normalized=False))
        gsbasis.append(basislst)

    return gsbasis

# 1d pbc Hubbard parameters
L = 10; U = 8; t = 1

mol = hubbard.HubbardMole(lx=L, ly=1, t=t, U=U)
mol.nelectron = L
mf = hubbard.RHF_SpinHubbard(mol)
mf.pbc = True
mf.kernel()

fcisolver = FCI(mf)
e_fci = fcisolver.kernel(nroots=2)[0]

h1e = mf.get_hcore()
eri = mf._eri

frag1 = [[[x] for x in range(L)]]
dmet_hf = DMET(mf, frag1, hubbard.DMET_Hubbard_SCF)
dmet_ci = DMET(mf, frag1, hubbard.DMET_Hubbard_FCI)

dmet_hf.chempot_cycle()
dmet_ci.chempot_cycle()

print(dmet_ci.dmets[0].mu_glob)
print("DMET Energy : ", dmet_ci.kernel_1shot()[0])

cps = CPSMethod(dmet_hf, dmet_ci, mf)
print(cps.get_ground_state()[0])
proj = cps.get_projectors()[0]

print(proj)

ci0 = cps.ci_slater([mf.mo_coeff[:,:L//2], mf.mo_coeff[:,:L//2]])

def func(x):
    p0, pu, pd, pud= x
    p = np.array([[p0,pu],[pd,pud]])
    return gwfexact_1site(ci0, L, L, p, h1e, eri)

def func_deriv(x):
    p0, pu, pd, pud= x
    p = np.array([[p0,pu],[pd,pud]])
    return gwfexact_grad_1site(ci0, L, L, p, h1e, eri)

ci_g = ci_cps(L, (L//2,L//2), ci0, 
    [proj], [[[i] for i in range(L)]], normalized=False)
S0 = np.dot(ci_g.reshape(-1), ci_g.reshape(-1))
proj *= S0**(-1/(2*L))

e_unopt_gwf = direct_spin1.energy(h1e, eri, ci_g, L, L) / S0

x0 = np.array(list(proj.reshape(-1)))

def _constraint(ci0, norb, nelec, proj, h1e, eri):
    fraglst_full = [[[i] for i in range(norb)]]
    ci_g = ci_cps(norb, (nelec//2,nelec//2), ci0, 
    [proj], fraglst_full, normalized=False)
    S = np.dot(ci_g.reshape(-1), ci_g.reshape(-1))
    return S-1

def constraint(x):
    p0, pu, pd, pud = x
    p = np.array([[p0,pu],[pd,pud]])
    return _constraint(ci0, L, L, p, h1e, eri)

cons = {"type": "eq", "fun": constraint}
res = minimize(func, x0, jac=func_deriv, constraints=cons)

converged SCF energy = 7.05572809000085


<>:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
/var/folders/9v/yn090wns46z2n124qdlklsw80000gn/T/ipykernel_2198/10851946.py:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if ci0 is 0: ci0 = np.random.random((num_string, num_string))
Initialize <dmet.model.hubbard.HubbardMole object at 0x11fae85b0> in RHF_SpinHubbard-RHF object of <class 'dmet.model.hubbard.RHF_SpinHubbard'>


converged SCF energy = -0.286572004444903
converged SCF energy = -0.258729336624559
converged SCF energy = 0.749429241478837
converged SCF energy = 0.704911651377252
converged SCF energy = 0.705572821441661
converged SCF energy = 0.705572809000552
2.559999999999941
DMET Energy :  -3.411223818645176
CPS Energy =  -2.3057706363292545
-2.3057706363292545
[[-0.52502039 -1.31314645]
 [-1.31314645 -0.52502039]]


$$
| \Psi_{GWF} \rangle = \prod\limits_{m = 1}^N \hat G_m | \Phi_0 \rangle
$$

in which $| \Phi_0 \rangle$ is the Hartree-Fock wave-function, and the Gutzwiller projector, if the $m$ represents the single site, is parameterized as

$$
\hat G_m = g_{\uparrow \downarrow} | \uparrow \downarrow \rangle \langle \uparrow \downarrow | + g_{\uparrow} | \uparrow \rangle \langle \uparrow | + g_{\downarrow} | \downarrow \rangle \langle \downarrow | + g_0 | \mathrm{vac} \rangle \langle \mathrm{vac} \rangle
$$

the coefficients below corresponds to $(g_0, g_{\uparrow}, g_{\downarrow}, g_{\uparrow \downarrow})$, respectively.

Notice: the coefficients are acquired through Full-CI vectors of 1-dimensional PBC hubbard model with $L=6$ or $L=10$.

Maybe GWF coefficient directly from DMET results can be used for complicated systems.


In [12]:
print("Unoptimized GWF Energy = ", e_unopt_gwf)
print("DMET GWF Coefficients: ", x0)
print("Optimized GWF Energy = ", res.fun)
print("Optimized GWF Coefficients: ", res.x)    # the
print("FCI Energy = ", e_fci)
print("HF Energy = ", mf.e_tot)

Unoptimized GWF Energy =  -2.3057706363291754
DMET GWF Coefficients:  [-0.4971562  -1.24345437 -1.24345437 -0.4971562 ]
Optimized GWF Energy =  -2.370239775058015
Optimized GWF Coefficients:  [-0.54679843 -1.23494136 -1.23494134 -0.54679843]
FCI Energy =  [-3.31499673 -2.72095332]
HF Energy =  7.055728090000853
